<a href="https://colab.research.google.com/github/Gwilson337/Project-4/blob/main/FinalProject/NFL_Fantasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from pathlib import Path


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [3]:
#Create file paths
csv_path1 = Path("/content/FantasyPros_2019.csv")
csv_path2 = Path("/content/FantasyPros_2020.csv")
csv_path3 = Path("/content/FantasyPros_2021.csv")
csv_path4 = Path("/content/FantasyPros_2022.csv")
csv_path5 = Path("/content/FantasyPros_2023.csv")

# Create DataFrames
df1 = pd.read_csv(csv_path1)
df2 = pd.read_csv(csv_path2)
df3 = pd.read_csv(csv_path3)
df4 = pd.read_csv(csv_path4)
df5 = pd.read_csv(csv_path5)

# #Join DataFrames
# all_df = pd.concat([df1, df2, df3, df4, df5], axis = 1)
# all_df.head()
# # all_df.shape

In [4]:
df1 = df1[["#", "Player", "Pos", "AVG", "TTL"]].rename(columns={"#" : "2019_rank", "AVG" : "2019_AVG", "TTL" : "2019_TTL"})
df2 = df2[["#", "Player", "AVG", "TTL"]].rename(columns={"#" : "2020_rank", "AVG" : "2020_AVG", "TTL" : "2020_TTL"})
df3 = df3[["#", "Player", "AVG", "TTL"]].rename(columns={"#" : "2021_rank", "AVG" : "2021_AVG", "TTL" : "2021_TTL"})
df4 = df4[["#", "Player", "AVG", "TTL"]].rename(columns={"#" : "2022_rank", "AVG" : "2022_AVG", "TTL" : "2022_TTL"})
df5 = df5[["Player", "TTL"]].rename(columns={"TTL" : "2023_TTL"})

all_df = pd.merge(df1, df2, on="Player", how="inner")
all_df = pd.merge(all_df, df3, on="Player", how="inner")
all_df = pd.merge(all_df, df4, on="Player", how="inner")
all_df = pd.merge(all_df, df5, on="Player", how="inner")

all_df

,2019_rank,Player,Pos,2019_AVG,2019_TTL,2020_rank,2020_AVG,2020_TTL,2021_rank,2021_AVG,2021_TTL,2022_rank,2022_AVG,2022_TTL,2023_TTL
0,1,Lamar Jackson,QB,28.1,421.7,10,22.8,341.8,19,21.1,253.0,19,20.3,243.1,338.2
1,2,Christian McCaffrey,RB,22.2,355.2,222,24.5,73.4,191,12.9,90.5,14,16.0,271.4,324.3
2,3,Dak Prescott,QB,21.8,348.8,91,27.7,138.6,8,20.7,330.6,28,17.8,213.6,351.8
3,4,Jameis Winston,QB,21.0,335.4,573,0.6,2.4,127,17.2,120.4,292,16.3,48.9,15.0
4,5,Russell Wilson,QB,20.9,333.6,6,23.3,372.8,20,17.8,248.8,22,15.7,235.8,264.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,565,Keith Kirkwood,WR,0.0,0.0,597,1.3,1.3,602,0.6,1.7,593,0.6,1.8,9.7
273,642,Tim Boyle,QB,0.0,-0.1,717,-0.1,-0.9,377,7.6,30.3,718,-0.9,-0.9,14.9
274,643,Nick Mullens,QB,-0.2,-0.3,109,12.8,128.3,498,9.9,9.9,468,3.2,12.8,74.7
275,648,Ray-Ray McCloud III,WR,-0.3,-2.0,466,0.8,12.2,388,1.7,27.2,311,2.6,44.1,16.5


In [ ]:
all_df = pd.merge(df1,df2,how="left",on="#")
all_df

In [ ]:
pd.merge??

In [ ]:
#Look for values equal to dash
all_df[all_df["17"]=="-"]


In [ ]:
#Replace missing values with zero
all_df.fillna(0, inplace=True)
all_df


In [ ]:
#Replace "bye weeks" with zero and check data types
all_df = all_df.replace("BYE", 0)
all_df = all_df.replace("-", 0)
# all_df.dtypes


In [ ]:
#Create list to store columns that should have numeric values
val_columns = ["#", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10",
               "11", "12", "13", "14", "15", "16", "17", "18"]
#Convert columns to float
all_df[val_columns] = all_df[val_columns].astype(float)
all_df.dtypes


In [ ]:
player_df = all_df[['Team','Player']]

In [ ]:
all_df = all_df.drop(columns=['Player'])
all_df.columns


In [ ]:
#all_df = all_df.drop(columns=['Team'])

In [ ]:
all_df.head()

In [ ]:
y = all_df['TTL']
X = all_df.drop(columns='TTL')



In [ ]:
X.head()

In [ ]:
y[:5]

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_scaler = scaler.fit(X_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)

In [ ]:
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
all_df.shape

In [ ]:
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=27))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu"))


In [ ]:
nn_model.compile(loss="mean_absolute_error", optimizer="adam", metrics=["accuracy"])

In [ ]:
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


In [ ]:
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)